In [ ]:
#!pip install optuna

In [1]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
import numpy as np

# Machine learning models that we'll use
import xgboost as xgb
import lightgbm as lgbm
#from sklearn.linear_model import Ridge
import catboost as cat
# optuna
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: Future

In [2]:
# We set as_frame parameter to True and access the return object's "frame"
# attribute to get the dataset as pandas dataframe.

df = fetch_california_housing(as_frame=True)["frame"]
print(df.shape)
df.head()

(20640, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy = train_test_split(df.drop(columns=['MedHouseVal']), df.MedHouseVal, test_size=0.15, shuffle=True, random_state=42)

In [4]:
from sklearn.metrics import mean_squared_error

# instantiating the model
cat_model = cat.CatBoostRegressor(verbose=False)
cat_model.fit(trainX, trainy)
y_pred_cat = cat_model.predict(testX)

xgb_model = xgb.XGBRegressor()
xgb_model.fit(trainX, trainy, verbose=False)
y_pred_xgb = xgb_model.predict(testX)

lgbm_model = lgbm.LGBMRegressor()
lgbm_model.fit(trainX, trainy, verbose=-1)
y_pred_lgbm = lgbm_model.predict(testX)

# combining predictions by taking simple average using numpy
y_pred_final = np.mean([y_pred_cat, y_pred_xgb, y_pred_lgbm], axis=0)

# let's calculate mse
mse = mean_squared_error(testy, y_pred_lgbm)

print(f"Simple Average Ensemble's MSE: {mse}") 
#Simple Average Ensemble's MSE: 0.20581131648521198

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Simple Average Ensemble's MSE: 0.21066428139566207


In [9]:
np.zeros(5).mean()

0.0

In [10]:
from sklearn.model_selection import KFold
import optuna

def objective(trial):
  STEP_SIZE = 1

  weights = []
  all_models_predictions = []

  # we'll use a variable for setting upper limit for suggested value
  # since we intend to update it after each weight suggestion
  upper_limit = 100

  w_cat = trial.suggest_int("w_cat", 0, upper_limit, step=STEP_SIZE)
  weights.append(w_cat)

  # Update upper limit to 100 - all the previous weights combined, which in this case is just w_ridge
  # WHY? well because we want to keep our sum of all weights equal to 100
  # and this is one way of ensuring that!
  upper_limit -= sum(weights)
  upper_limit = upper_limit

  w_xgb = trial.suggest_int("w_xgb", 0, upper_limit, step=STEP_SIZE)
  weights.append(w_xgb)

  # for the final weight we won't use optuna, rather we'll manually set it equal
  # to whatever value remains after subtracting the sum of suggested weight values from 100
  # This will also make sure that the sum of all weights remains equal to 100.
  w_lgbm = 100 - sum(weights)
  weights.append(w_lgbm)

  # Just as a sanity check, we'll check that the sum of all weights is equal to 100
  weights_sum = sum(weights)

  if weights_sum != 100:
    raise Exception(f"Weights sum must be equal to 100. Instead {weights_sum} was encountered!")
  
  # We'll use the default parameter values for all our models
  cat_model = cat.CatBoostRegressor()
  cat_model.fit(trainX, trainy)
  y_pred_cat = cat_model.predict(testX)
  all_models_predictions.append(y_pred_cat)

  xgb_model = xgb.XGBRegressor()
  xgb_model.fit(trainX, trainy)
  y_pred_xgb = xgb_model.predict(testX)
  all_models_predictions.append(y_pred_xgb)

  lgbm_model = lgbm.LGBMRegressor()
  lgbm_model.fit(trainX, trainy, verbose=-1)
  y_pred_lgbm = lgbm_model.predict(testX)
  all_models_predictions.append(y_pred_lgbm)

  # let's take the weighted average of the predictions using numpy
  y_pred_final = np.average(all_models_predictions, weights=weights, axis=0)
  # computing our metric i.e. MSE
  mse = mean_squared_error(testy, y_pred_final)

  return mse

In [11]:
study = optuna.create_study(study_name="optimizing weights", direction="minimize")
study.optimize(objective, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-08-02 01:26:10,666] A new study created in memory with name: optimizing weights


Learning rate set to 0.064991
0:	learn: 1.1134528	total: 26.9ms	remaining: 26.9s
1:	learn: 1.0764030	total: 55ms	remaining: 27.5s
2:	learn: 1.0409692	total: 74.5ms	remaining: 24.8s
3:	learn: 1.0083718	total: 85ms	remaining: 21.2s


4:	learn: 0.9788923	total: 192ms	remaining: 38.3s
5:	learn: 0.9515839	total: 215ms	remaining: 35.6s
6:	learn: 0.9247004	total: 221ms	remaining: 31.4s
7:	learn: 0.9001281	total: 227ms	remaining: 28.2s
8:	learn: 0.8778985	total: 236ms	remaining: 26s
9:	learn: 0.8559244	total: 246ms	remaining: 24.4s
10:	learn: 0.8370810	total: 276ms	remaining: 24.8s
11:	learn: 0.8193220	total: 283ms	remaining: 23.3s
12:	learn: 0.8039389	total: 304ms	remaining: 23.1s
13:	learn: 0.7893027	total: 349ms	remaining: 24.5s
14:	learn: 0.7759102	total: 361ms	remaining: 23.7s
15:	learn: 0.7622046	total: 374ms	remaining: 23s
16:	learn: 0.7511464	total: 389ms	remaining: 22.5s
17:	learn: 0.7402253	total: 404ms	remaining: 22s
18:	learn: 0.7304534	total: 415ms	remaining: 21.4s
19:	learn: 0.7192693	total: 427ms	remaining: 20.9s
20:	learn: 0.7104842	total: 445ms	remaining: 20.8s
21:	learn: 0.7020815	total: 455ms	remaining: 20.2s
22:	learn: 0.6950149	total: 475ms	remaining: 20.2s
23:	learn: 0.6869116	total: 495ms	remaining

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
[I 2023-08-02 01:26:25,624] Trial 0 finished with value: 0.19715595834169064 and parameters: {'w_cat': 75, 'w_xgb': 10}. Best is trial 0 with value: 0.19715595834169064.


Learning rate set to 0.064991
0:	learn: 1.1134528	total: 11.7ms	remaining: 11.7s
1:	learn: 1.0764030	total: 16.9ms	remaining: 8.43s
2:	learn: 1.0409692	total: 27.7ms	remaining: 9.2s
3:	learn: 1.0083718	total: 39.6ms	remaining: 9.87s
4:	learn: 0.9788923	total: 48.1ms	remaining: 9.58s
5:	learn: 0.9515839	total: 52.8ms	remaining: 8.75s
6:	learn: 0.9247004	total: 61.1ms	remaining: 8.67s
7:	learn: 0.9001281	total: 68.4ms	remaining: 8.49s
8:	learn: 0.8778985	total: 74.6ms	remaining: 8.21s
9:	learn: 0.8559244	total: 83.8ms	remaining: 8.29s
10:	learn: 0.8370810	total: 88.7ms	remaining: 7.97s
11:	learn: 0.8193220	total: 97.4ms	remaining: 8.02s
12:	learn: 0.8039389	total: 103ms	remaining: 7.84s
13:	learn: 0.7893027	total: 108ms	remaining: 7.61s
14:	learn: 0.7759102	total: 113ms	remaining: 7.42s
15:	learn: 0.7622046	total: 118ms	remaining: 7.24s
16:	learn: 0.7511464	total: 122ms	remaining: 7.07s
17:	learn: 0.7402253	total: 127ms	remaining: 6.95s
18:	learn: 0.7304534	total: 133ms	remaining: 6.85s


...

Learning rate set to 0.064991
0:	learn: 1.1134528	total: 9.31ms	remaining: 9.3s
1:	learn: 1.0764030	total: 15.9ms	remaining: 7.94s
2:	learn: 1.0409692	total: 22.6ms	remaining: 7.52s
3:	learn: 1.0083718	total: 29.6ms	remaining: 7.38s
4:	learn: 0.9788923	total: 36.6ms	remaining: 7.29s
5:	learn: 0.9515839	total: 42ms	remaining: 6.96s
6:	learn: 0.9247004	total: 49.1ms	remaining: 6.96s
7:	learn: 0.9001281	total: 60.4ms	remaining: 7.49s
8:	learn: 0.8778985	total: 67.8ms	remaining: 7.46s
9:	learn: 0.8559244	total: 76.2ms	remaining: 7.55s
10:	learn: 0.8370810	total: 83.4ms	remaining: 7.5s
11:	learn: 0.8193220	total: 89.5ms	remaining: 7.37s
12:	learn: 0.8039389	total: 95.4ms	remaining: 7.24s
13:	learn: 0.7893027	total: 102ms	remaining: 7.19s
14:	learn: 0.7759102	total: 110ms	remaining: 7.25s
15:	learn: 0.7622046	total: 117ms	remaining: 7.21s
16:	learn: 0.7511464	total: 126ms	remaining: 7.31s
17:	learn: 0.7402253	total: 134ms	remaining: 7.33s
18:	learn: 0.7304534	total: 142ms	remaining: 7.35s
19

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
[I 2023-08-02 01:29:41,293] Trial 18 finished with value: 0.19826377279871282 and parameters: {'w_cat': 86, 'w_xgb': 4}. Best is trial 12 with value: 0.19656207826532104.


Learning rate set to 0.064991
0:	learn: 1.1134528	total: 9.43ms	remaining: 9.42s
1:	learn: 1.0764030	total: 19.2ms	remaining: 9.56s
2:	learn: 1.0409692	total: 25.6ms	remaining: 8.52s
3:	learn: 1.0083718	total: 32.3ms	remaining: 8.05s
4:	learn: 0.9788923	total: 40.8ms	remaining: 8.11s
5:	learn: 0.9515839	total: 47.6ms	remaining: 7.88s
6:	learn: 0.9247004	total: 54.6ms	remaining: 7.75s
7:	learn: 0.9001281	total: 62.1ms	remaining: 7.7s
8:	learn: 0.8778985	total: 68.5ms	remaining: 7.54s
9:	learn: 0.8559244	total: 75.9ms	remaining: 7.52s
10:	learn: 0.8370810	total: 83.7ms	remaining: 7.52s
11:	learn: 0.8193220	total: 90ms	remaining: 7.41s
12:	learn: 0.8039389	total: 97.1ms	remaining: 7.38s
13:	learn: 0.7893027	total: 113ms	remaining: 7.94s
14:	learn: 0.7759102	total: 120ms	remaining: 7.87s
15:	learn: 0.7622046	total: 126ms	remaining: 7.76s
16:	learn: 0.7511464	total: 133ms	remaining: 7.67s
17:	learn: 0.7402253	total: 139ms	remaining: 7.6s
18:	learn: 0.7304534	total: 145ms	remaining: 7.49s
19

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
[I 2023-08-02 01:29:52,685] Trial 19 finished with value: 0.1965951392694841 and parameters: {'w_cat': 63, 'w_xgb': 25}. Best is trial 12 with value: 0.19656207826532104.


Number of finished trials: 20
Best trial: {'w_cat': 57, 'w_xgb': 19}


In [22]:
cat_model = cat.CatBoostRegressor()
cat_model.fit(trainX, trainy)
y_pred_cat = cat_model.predict(testX)

xgb_model = xgb.XGBRegressor()
xgb_model.fit(trainX, trainy)
y_pred_xgb = xgb_model.predict(testX)

lgbm_model = lgbm.LGBMRegressor()
lgbm_model.fit(trainX, trainy, verbose=-1)
y_pred_lgbm = lgbm_model.predict(testX)

params = study.best_trial.params
y_pred = (params['w_cat']*y_pred_cat+params['w_xgb']*y_pred_xgb+(100-params['w_cat']-params['w_xgb'])*y_pred_lgbm)/100

Learning rate set to 0.064991
0:	learn: 1.1134528	total: 7.25ms	remaining: 7.24s
1:	learn: 1.0764030	total: 18.5ms	remaining: 9.23s
2:	learn: 1.0409692	total: 23.8ms	remaining: 7.9s
3:	learn: 1.0083718	total: 28.9ms	remaining: 7.21s
4:	learn: 0.9788923	total: 35.4ms	remaining: 7.04s
5:	learn: 0.9515839	total: 51.2ms	remaining: 8.49s
6:	learn: 0.9247004	total: 67.1ms	remaining: 9.52s
7:	learn: 0.9001281	total: 73.1ms	remaining: 9.06s
8:	learn: 0.8778985	total: 80ms	remaining: 8.81s


9:	learn: 0.8559244	total: 189ms	remaining: 18.7s
10:	learn: 0.8370810	total: 195ms	remaining: 17.6s
11:	learn: 0.8193220	total: 215ms	remaining: 17.7s
12:	learn: 0.8039389	total: 222ms	remaining: 16.8s
13:	learn: 0.7893027	total: 252ms	remaining: 17.8s
14:	learn: 0.7759102	total: 259ms	remaining: 17s
15:	learn: 0.7622046	total: 268ms	remaining: 16.5s
16:	learn: 0.7511464	total: 273ms	remaining: 15.8s
17:	learn: 0.7402253	total: 281ms	remaining: 15.3s
18:	learn: 0.7304534	total: 290ms	remaining: 15s
19:	learn: 0.7192693	total: 303ms	remaining: 14.9s
20:	learn: 0.7104842	total: 322ms	remaining: 15s
21:	learn: 0.7020815	total: 330ms	remaining: 14.7s
22:	learn: 0.6950149	total: 342ms	remaining: 14.5s
23:	learn: 0.6869116	total: 349ms	remaining: 14.2s
24:	learn: 0.6779583	total: 365ms	remaining: 14.2s
25:	learn: 0.6717269	total: 379ms	remaining: 14.2s
26:	learn: 0.6635544	total: 387ms	remaining: 13.9s
27:	learn: 0.6579794	total: 394ms	remaining: 13.7s
28:	learn: 0.6513783	total: 440ms	rema

/Users/yukaisun/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [24]:
# let's calculate mse
mse = mean_squared_error(testy, y_pred)
print(f"Simple Average Ensemble's MSE: {mse}") 

Simple Average Ensemble's MSE: 0.19656207805707848


In [30]:
#pd.DataFrame({'predy':y_pred, 'testy':pd.Series(testy), 'avg_predy': y_pred_final}).head(20)